In [ ]:
from ase import Atoms
import ase.units as units
from ase.io import read
from ase.build import bulk
from ase.calculators.espresso import Espresso

import os
import dotenv
dotenv.load_dotenv()
PWSCF_COMMAND = os.environ.get("PWSCF_COMMAND")
PPDIR = os.environ.get("PSEUDOPOTENTIALS")

outdir = "./results"
if not os.path.exists(outdir):
    os.mkdir(outdir)

# Create BCC Fe bulk and slab
bulk_Fe = bulk('Fe', 'bcc', a=2.87)  # Adjust 'a' for lattice parameter
slab_Fe = surface(bulk_Fe, (1, 1, 1), layers=5)  # Change orientation and layers as needed
slab_Fe.center(vacuum=10.0, axis=2)  # Add vacuum spacing

# Apply constraints if necessary
mask = [atom.tag > 2 for atom in slab_Fe]
slab_Fe.set_constraint(FixAtoms(mask=mask))

k_points = 4  # k-points grid, adjust as needed
pseudopotentials = {'Fe': 'Fe.pbe-nd-rrkjus.UPF'}  # Example, change as needed

outdir = "results"

calc_params = {
    'calculation': 'vc-relax',
    'tprnfor': True,
    'tstress': True,
    'input_data': {
        'system': {
            'ecutwfc': 30 * units.eV,  # PW cutoff
            'ecutrho': 240 * units.eV,  # Charge cutoff
        },
        'electrons': {
            'diagonalization': 'david',
            'mixing_beta': 0.5,
            'conv_thr': 1e-07,
        }
    },
    'pseudopotentials': pseudopotentials,
    'kpts': (k_points, k_points, 1),  # k-points grid, adjust as needed
    'parallel': 'all',
    'directory': outdir,  # Custom directory for calculation files
    'label': "Fe",  # Prefix for the filenames
    'logfile': "Fe.log",  # Logfile name
    'command': "mpirun -np 16 " + PWSCF_COMMAND + " -in Fe.pwi > Fe.pwo",
    'pseudo_dir': PPDIR,
}

# Set up Quantum Espresso calculator
calc = Espresso(**calc_params)  # k-point grid

# Attach calculator
slab_Fe.set_calculator(calc)

# Run calculation
total_energy_slab = slab_Fe.get_potential_energy()

# Bulk energy per atom
bulk_Fe.set_calculator(calc)
total_energy_bulk = bulk_Fe.get_potential_energy()
energy_per_atom_bulk = total_energy_bulk / len(bulk_Fe)

# Compute surface free energy
n_atoms = len(slab_Fe)
surface_area = slab_Fe.get_surface_area()
surface_energy = (total_energy_slab - n_atoms * energy_per_atom_bulk) / (2 * surface_area)

print("Surface Free Energy of BCC Fe:", surface_energy, "eV/A^2")